# Transcribe Recordings
paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [334]:
import requests
import pandas as pd
import os
import shutil
import io
import json
import math

from requests.utils import quote

# audio
from pydub import AudioSegment

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *

from lib_henryk.transcription import transcription

## Params

In [346]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

WEBHOOKS_TOKEN_ID = '2660666c-d957-4485-9a95-5e264563a022'

## Load Dataset

In [336]:
df = pd.read_csv(FILE_AUDIO_STATS_CSV)
df_transcription = df.copy()
df_transcription['transcription_id'] = ''

## Run Transcriptions

In [501]:
# load sync dataframe
if os.path.exists(FILE_TRANSCRIPTION_CSV):
    print(f'transcription log exists, reading it from {FILE_TRANSCRIPTION_CSV}')
    df_transcription = pd.read_csv(FILE_TRANSCRIPTION_CSV)

transcription log exists, reading it from ../../data/processed/henryk_audio_transcription.csv


In [502]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df_transcription, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    start=150, stop=200,
    verbose=False
)

processing [150]: Henryk 2022-12-16 Królik rozmawia z Puchatkiem i Prosiaczkiem o odbryknięciu Tygryska.m4a
b'{"detail":"Error processing request. You\'ll not be billed for this request."}'
submitted 0 recordings to the transcription service GoodTape


## Fetch Transcriptions JSONs

In [470]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

found 0 requests to fetch
0 transcriptions were processed


## Process JSONs to Transcriptions

In [500]:
# process json to text
transcription.process_transcriptions_json_to_text(
    df=df_transcription, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions=DIR_TRANSCRIPTIONS,
    verbose=False
)

processed [0]: Henryk 2022-05-26 Bajka o nowym samochodziku.txt
processed [1]: Henryk 2022-05-27 Bajka o Eryku r1.txt
processed [2]: Henryk 2022-05-28 Bajka o swince peppie i domu rebeki.txt
processed [3]: Henryk 2022-05-30 Bajka o parku dinozaurow.txt
processed [4]: Henryk 2022-05-31 Bajka o drużynie koszykowki.txt
processed [5]: Henryk 2022-06-01 Bajka o bananach.txt
processed [6]: Henryk 2022-06-02 Bajka o kotkach i majówce.txt
processed [7]: Henryk 2022-06-03 Bajka o jeżyku cyprianie.txt
processed [8]: Henryk 2022-06-04 W starym gaju.txt
processed [9]: Henryk 2022-06-05 Bajka o okruchach słońca.txt
processed [10]: Henryk 2022-06-06 Bajka po peppie i perfumach.txt
processed [11]: Henryk 2022-06-07 Bajka o robotach drogowych.txt
processed [12]: Henryk 2022-06-08 Bajka o gimnastyce.txt
processed [13]: Henryk 2022-06-09 Bajka o peppie i misiu.txt
processed [14]: Henryk 2022-06-10 Bajka o czkawce.txt
processed [15]: Henryk 2022-06-13 Bajka o sloneczku.txt
processed [16]: Henryk 2022

## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [493]:
# cleanup stale transcription ids
transcription.cleanup_missing_transcriptions(df_transcription)

# save transcription log
df_transcription.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

there are 146 processed and valid transcriptions


In [ ]:
# df_transcription = pd.read_csv(FILE_TRANSCRIPTION_CSV)
# df_transcription = df_transcription_backup.copy()